In [7]:
%matplotlib inline
import matplotlib.pylab as plt 
import numpy as np 
import pandas as pd 
import geopandas as gpd
import rasterio 
import rasterio.mask
import os 
import sys 
from tqdm.notebook import tqdm 




import requests
from requests.auth import HTTPBasicAuth 
import zipfile 



from bs4 import BeautifulSoup
from datetime import datetime 
from datetime import date 

sys.path.append("/Users/mishaklein/Documents/redcross_510")
from redcross510_utils import * 



# Enter the stuff that changes over here 

In [41]:
today = str(date.today())
download_dir = './data_download'
zonal_stats_file_name = f"PHL_admin3_zonal_statistics_{today.replace('-','_')}.csv"

In [25]:
dict_typhoones={'bopha':'2012-12-04 04:45:00',
                'durian':'2006-11-30 00:00:00',
                'fengshen':'2008-06-21 12:00:00',
                'ketsana':'2009-09-26 00:00:00',
                'washi':'2011-12-16 00:00:00',
                'haiyan':'2013-11-08 00:00:00',
                'hagupit':'2014-12-06 23:00:00',
                'haima':'2016-10-19 23:00:00',
                'nock-ten':'2016-12-25 18:00:00',
                'mangkhut':'2018-09-15 01:40:00',
                'kammuri':'2019-12-02 20:00:00',
                'phanfone':'2019-12-24 00:00:00',
                'vongfong':'2020-05-14 00:00:00',
                'molave':'2020-10-25 18:10:00',
                'goni':'2020-11-01 05:00:00'}

# Downloading data from the web 

In [28]:
def get_NASA_EOSDIS_file(year,month,day, username, password):
    '''
    
    Access the download page of the Earth Observing System Data and Information System (EOSDIS)
    and get return the download link for the daily data (.zip)
    '''
    # Go to page 
    baseURL = "https://arthurhouhttps.pps.eosdis.nasa.gov/gpmallversions/V06/"
    page = requests.get(url = os.path.join(baseURL, year, month, day, 'gis'),
                       auth=HTTPBasicAuth(username, password))
    
    
    # BeautifulSoup package has very convinient ways to search through such content 
    soup = BeautifulSoup(page.content, 'html.parser')

    for a in soup.find_all('a'):
        link = a.get('href')
        # We know the prefix of the file name and it's extension
        if link.startswith("3B-DAY-GIS") and link.endswith(".zip"):
            download_file = link 
    return os.path.join(baseURL, year, month, day, 'gis', download_file)




def download_rainfall_typhoons_phillipines(typhoons_of_interest = {}, 
                                           download_dir = './data_download',
                                           username = "akliludin@gmail.com",
                                           password = "akliludin@gmail.com"):
    '''
    input a dictionary with keys of typhoon names and values the dates in "year-month-day hour:minutes:seconds" format
    this will download the daily rainfall data, and unzip the obtained zip-archives 
    '''
    
    baseURL = "https://arthurhouhttps.pps.eosdis.nasa.gov/gpmallversions/V06/"
    
    download_links = []
    for typhoon in tqdm(typhoons_of_interest): 
        
        # use datetime object to handle the date, month, year, wich we'll need to get the final download link
        date_of_typhoon = datetime.strptime(typhoons_of_interest[typhoon], '%Y-%m-%d %H:%M:%S')
        
        year = str(date_of_typhoon.year) 
        if date_of_typhoon.month < 10:
            month = "0" + str(date_of_typhoon.month)
        else: 
            month = str(date_of_typhoon.month)

        if date_of_typhoon.day < 10:
            day = "0" + str(date_of_typhoon.day)
        else: 
            day = str(date_of_typhoon.day)
        
        
        # get the complete link to the zipfiles to be downloaded
        fullURL = get_NASA_EOSDIS_file(year,month,day, username, password)

        # download files 
        download_file = os.path.join(download_dir, fullURL.split('/')[-1])
        
        # if not already donwloaded: 
        if not os.path.join(download_dir, typhoon) + ".zip":
            download_files_url(url = fullURL, username = username, password = password, 
                               path_download_file = download_file)

            # unzip into new folder 
            output_folder = os.path.join(download_dir, typhoon)
            if not os.path.exists( output_folder):
                os.makedirs( output_folder )


            new_name = os.path.join(download_dir, typhoon) + ".zip"
            os.rename(download_file, new_name)
            extract_zip_archive(zip_archive = new_name, destination_dir = output_folder)
            
        
        # create log file (to know what happened)
        download_links.append(fullURL)
    
    # create log file (to know what happened)
    today = str(date.today())
    log_file_name = os.path.join(download_dir, '_'.join(['info_downloads', today.replace('-','_')]) + '.csv')
    logFile = pd.DataFrame(typhoons_of_interest, columns = ["typhoon_name", "date"])
    logFile['downloaded_from'] = download_links 
    logFile['downloaded_to'] = download_dir
    logFile.to_csv(log_file_name, index = False)

In [29]:
download_rainfall_typhoons_phillipines(typhoons_of_interest = dict_typhoons)

  0%|          | 0/15 [00:00<?, ?it/s]

# Zonal Statistics 

In [30]:
file_shapes = "/Users/mishaklein/Documents/redcross_510/shapefiles/phl_admin3_version2/phl_admin3_version2.shp"

shapeData = gpd.read_file(file_shapes)
shapeData 

,gridid,glat,glon,geometry
0,PH01280100,18.453,120.919,POINT (120.91941 18.45339)
1,PH01280200,18.267,120.613,POINT (120.61290 18.26725)
2,PH01280300,17.906,120.509,POINT (120.50897 17.90596)
3,PH01280400,18.490,120.750,POINT (120.74967 18.48975)
4,PH01280500,18.038,120.584,POINT (120.58423 18.03775)
...,...,...,...,...
5672,PH18462200,9.944,123.075,POINT (123.07455 9.94447)
5673,PH18462300,9.291,123.161,POINT (123.16082 9.29122)
5674,PH18462400,10.279,123.331,POINT (123.33148 10.27914)
5675,PH18462400,10.343,123.293,POINT (123.29275 10.34337)


I do this to check the column name that has either the region's name or the p-code 

### perform zonal statistics for every typhoon (one TIFF file) and merge together into one final output table 

In [48]:
shapefile = "/Users/mishaklein/Documents/redcross_510/shapefiles/phl_admin3_version2/phl_admin3_version2.shp"



typhoonData = pd.DataFrame()
for typhoon in tqdm(dict_typhoons): 
    data_dir = os.path.join(download_dir, typhoon)
    filenames = [f for f in os.listdir(data_dir) if f.endswith('total.accum.tif')]
    assert len(filenames) == 1
    rasterfile =  os.path.join(download_dir, typhoon, filenames[0])
    
    oneTyphoon = zonal_statistics(rasterfile,shapefile, pcodeKey = "gridid")
    oneTyphoon['typhoon_name'] = typhoon 
    
    typhoonData = pd.concat([typhoonData, oneTyphoon])
    

typhoonData.to_csv(zonal_stats_file_name, index = False)
print(f"wrote file: {zonal_stats_file_name}")
typhoonData

  0%|          | 0/15 [00:00<?, ?it/s]

wrote file: PHL_admin3_zonal_statistics_2021_05_13.csv


,pcode,value,typhoon_name
0,PH01280100,2.0,bopha
1,PH01280200,3.0,bopha
2,PH01280300,7.0,bopha
3,PH01280400,0.0,bopha
4,PH01280500,6.0,bopha
...,...,...,...
5672,PH18462200,305.0,goni
5673,PH18462300,52.0,goni
5674,PH18462400,10.0,goni
5675,PH18462400,17.0,goni
